# Enunciado 

Un resorte helicoidal de compresión de alambre de piano con vida infinita se necesita para resistir una carga dinámica que varía de 5 a 20 lb a 5 Hz, 
en tanto que la deflexión del extremo varía de ½ a 2 in. 

> Debido a consideraciones de ensamble, 

1. la altura sólida no debe exceder 1 in y la longitud libre no puede ser mayor que 4 in. 

El fabricante de resortes tiene los siguientes dados en existencia: 0.069, 0.071, 0.080, 0.085, 0.090, 0.095, 0.105 y 0.112 in.

### Import necessary libraries

In [187]:
import pandas as pd
from sympy import Eq, solve, Symbol
import numpy as np
from colorama import Fore

In [188]:
### tools

k_b_func = lambda c: (4 * c + 2) / (4 * c - 3)

c_func = lambda a, b: (2 * a - b) / (4 * b) + pow(
        pow((2*a-b)/(4*b), 2) - 3*a/(4*b), 
        1/2
)

na_func = lambda d, G, Dm, k: (pow(d, 4) * G)/(8 * pow(Dm, 3) * k)

### Establecer datos conocidos

In [189]:
F_max = 20 #lb
F_min = 5 #lb
f = 5 # hz
L_f = 4 # in
y_max = 2 # in
y_min = 1 / 2 # in
nf = 1.5
'''factor a la fatiga'''
eta = 0.15 # epsilon
'''epsilon'''

d = np.array([0.069, 0.071, 0.080, 0.085, 0.090, 0.095, 0.105, 0.112], dtype=float)

diameter = d[-1]

### Selección del material

In [190]:
table_10_5 = pd.read_csv('tables/table_10_5.csv')
table_10_5.head()

,Material,Limite tension,limite torsion,Diámetro in,E M-psi,E GPa,G M-psi,G GPa
0,Alambre de piano A228,65-75,45-60,<0.032,29.5,203.4,12.00,82.7
1,Alambre de piano A228,65-75,45-60,0.033-0.063,29.0,200.0,11.85,81.7
2,Alambre de piano A228,65-75,45-60,0.064-0.125,28.5,196.5,11.75,81.0
3,Alambre de piano A228,65-75,45-60,>0.125,28.0,193.0,11.60,80.0
4,Resorte estirado duro A227,60-70,45-55,<0.032,28.8,198.6,11.70,80.7


In [191]:
# Search Cromo-vanadio A231	
count = 2
material  = table_10_5[table_10_5['Material'] == 'Alambre de piano A228'].\
    values[count][1:]

G = material[-2]

porcentaje_sut = int(material[1].split('-')[0])/100
material, porcentaje_sut, G

(array(['65-75', '45-60', '0.064-0.125', 28.5, 196.5, 11.75, 81.0],
       dtype=object),
 0.45,
 11.75)

### Encontrar Fuerza alternante y media

In [192]:
F_a = (F_max - F_min) / 2 
F_m = (F_max + F_min) / 2

print('Fuerza alternante', Fore.LIGHTBLUE_EX, F_a, 'lb', Fore.RESET)
print('Fuerza media', Fore.LIGHTBLUE_EX, F_m, 'lb', Fore.RESET)

Fuerza alternante  7.5 lb 
Fuerza media  12.5 lb 


### Encontrar k

In [193]:
k = F_max / y_max
print('indice de resorte k', Fore.LIGHTCYAN_EX, k, 'lb/ln', Fore.RESET)

indice de resorte k  10.0 lb/ln 


### Encontrar la resistencia ultima a la tracción

In [194]:
table_10_4 = pd.read_csv('tables/table_10_4.csv')
table_10_4.head()

,Material,ASTM num,exponente m,diametro in,A kpsi * in^m,Diametro mm,A Mpa * mm^m,Costo relativo,diametro bajo,diametro alto
0,Alambre de piano,A228,0.145,0.004-0.256,201,0.10-6.5,2211,2.6,0.004,0.256
1,Alambre T y R en aceite,A229,0.187,0.020-0.500,147,0.5-12.7,1 855,1.3,0.020,0.500
2,Alambre estirado duro,A227,0.190,0.028-0.500,140,0.7-12.7,1 783,1.0,0.028,0.500
3,Alambre al cromo- vanadio,A232,0.168,0.032-0.437,169,0.8-11.1,2 005,3.1,0.032,0.437
4,Alambre al cromo- silicio,A401,0.108,0.063-0.375,202,1.6-95,1974,40,0.063,0.375


In [195]:
seleccion = table_10_4[
    (table_10_4['ASTM num'] == 'A228' )
    & (diameter < table_10_4['diametro alto']) 
    & (diameter > table_10_4['diametro bajo'])
].values[0]

In [196]:
m = seleccion[2]
A = seleccion[4]
print(f'm = {m :,.3f} ', f'A = {A :,.2f} kpi * in^m')

S_ut = A / pow(diameter, m)
# print(S_ut)
print(f'S_ut = {S_ut :,.5f} kpsi')

m = 0.145  A = 201.00 kpi * in^m
S_ut = 276.09557 kpsi


### Encontrar la resistencia ultima al cortante y el modulo de ruptura por torsion

In [197]:
S_su = 0.67 * S_ut
print('Resistencia al cortante S_sy', Fore.LIGHTCYAN_EX, S_su, 'ksi', Fore.RESET)

Resistencia al cortante S_sy  184.98403435613903 ksi 


In [198]:
S_sy = porcentaje_sut * S_ut
print('Resistencia al cortante S_sy', Fore.LIGHTCYAN_EX, S_sy, 'ksi', Fore.RESET)

Resistencia al cortante S_sy  124.24300814964562 ksi 


### Método Sines

In [199]:
S_se = 35e3 # psi

### Calculo de C  de 5 a 12

In [200]:
alfa  = S_se / nf
print('alfa', Fore.LIGHTMAGENTA_EX, alfa, 'psi', Fore.RESET)

alfa  23333.333333333332 psi 


In [201]:
beta = 8 * F_a / (np.pi * pow(diameter, 2))
print('beta = ', Fore.LIGHTYELLOW_EX, beta, 'psi', Fore.RESET)

beta =   1522.5281545780801 psi 


In [202]:
c = c_func(alfa, beta)
print('c =', Fore.LIGHTGREEN_EX, c, Fore.RESET)

c =  14.004657239753158 


### Calculo del diámetro medio

In [203]:
Dm = c * diameter
print('Diámetro medio', Fore.LIGHTCYAN_EX,  Dm, 'in', Fore.RESET)

Diámetro medio  1.5685216108523536 in 


### Encontrar la fuerza solida

In [204]:
F_s = (1 + eta) * F_max
print('Fuerza solida')

Fuerza solida


### Encontrar numero de espiras activas

In [205]:
Na = na_func(diameter, G*1E6, Dm, k)
print('Numero de espiras activas: ', Fore.LIGHTCYAN_EX, Na, Fore.RESET)

Numero de espiras activas:   5.98891914949793 


### Calculo de longitud solida

In [206]:
L_s = (Na) * diameter
print('Longitud solida L s', Fore.LIGHTCYAN_EX, L_s , 'in', Fore.RESET)

Longitud solida L s  0.6707589447437682 in 


### Calculo de la longitud libre

In [207]:
L_f = F_s/k + L_s
print('longitud libre', Fore.LIGHTGREEN_EX, L_f, 'in', Fore.RESET)

longitud libre  2.970758944743768 in 


### Cálculos tipos

In [210]:
Di = Dm - diameter
print('Diámetro menor Di', Fore.LIGHTCYAN_EX, Di, 'in', Fore.RESET)

De = Dm + diameter
print('Diámetro mayor De', Fore.LIGHTCYAN_EX, De, 'in', Fore.RESET)

Diámetro menor Di  1.4565216108523535 in 
Diámetro mayor De  1.6805216108523537 in 


### Calculo de la deflection

In [ ]:
y_s = L_f - L_s
print('delta')

### Calculo de longitud critica

In [214]:
# relación de pandeo
pandeo_relation = 5.26 * Dm
print('L_f', Fore.LIGHTCYAN_EX, L_f, Fore.RESET, '5.26 * D_m', Fore.LIGHTCYAN_EX, pandeo_relation, Fore.RESET, L_f < pandeo_relation,)


L_f  2.970758944743768  5.26 * D_m  8.25042367308338  True


### Encontrar k_b

In [209]:
k_b = k_b_func(c)
print('indice de resorte k_b', Fore.LIGHTCYAN_EX, k_b, Fore.RESET)

indice de resorte k_b  1.094306474878205 
